# Grouping and Aggregation

## Contains:
### Mean by department_id
### Mean price of item by loyalty flag
### Creation of spending flag column
### Creation of frequency flag column
### Cleaning outliers from item prices

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#set up file path
path = r'C:\Users\krist\08.2023_InstacartBasketAnalysis'

In [3]:
#import dataframe from reading and check import
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02.Data', 'Prepared_Data', 'orders_products_merged_4.8_reading.pkl'))
df_ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,_exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,11.0,196,1,0,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy days,most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Least busy,Least busy days,average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,77,7,9.0,both,Mid-range product,Least busy,Least busy days,most orders,10,New customer


In [4]:
#check for shape
df_ords_prods_merge.shape

(32404859, 21)

### shape matches df exported from 4.8 reading: 324,004,859 rows 21 columns

In [5]:
#find aggregated mean of order_number grouped by department_id
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## Checked script from 4.4 for departments
### the lowest number of orders per department is 15.2 in alcohol (dept 5)
### these departments all have 15 orders frozen, alcohol, pets, dry goods pasta, meat seafood, and household (1,5,8,9,12,17)
### the highest number of orders per department is 22.9 in missing (dept 21), the next highest is in bulk (dept 10) 
### in the subset we didn't have all departments and the lowest number of orders was 11 in household (dept 17)
### in the subset we didn't have all departments and the highest number of orders was 19.4 in dairy eggs (dept 16)
### if we had only used the subset we would have had the wrong highs and low and the wrong departments 

In [6]:
#loyalty flag already exists because I imported the dataset from the reading exercise
df_ords_prods_merge['loyalty_flag'].head(25)

0         New customer
1         New customer
2         New customer
3         New customer
4         New customer
5         New customer
6         New customer
7         New customer
8         New customer
9         New customer
10    Regular customer
11    Regular customer
12    Regular customer
13    Regular customer
14    Regular customer
15        New customer
16        New customer
17        New customer
18    Regular customer
19    Regular customer
20    Regular customer
21    Regular customer
22    Regular customer
23    Regular customer
24    Regular customer
Name: loyalty_flag, dtype: object

In [7]:
#check to see if type of customer impacts average price of purchased products
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.386336
New customer,13.294670
Regular customer,12.495717


### on average loyal customers by products that are less expensive than new or regular customeres
### loyal customer product price mean = 10.39
### regular customer product price mean = 12.50
### new customer product price mean = 13.29

## Create new flag by first grouping by user and finding avg product price for each user then creating flag column

In [8]:
#create a new column with mean of product price for each user
df_ords_prods_merge['users_avg_product_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [9]:
#check for new column
df_ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,department_id,prices,_exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,users_avg_product_price
0,2539329,1,1,2,8,11.0,196,1,0,both,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,both,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,both,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,both,...,7,9.0,both,Mid-range product,Least busy,Least busy days,average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,both,...,7,9.0,both,Mid-range product,Least busy,Least busy days,most orders,10,New customer,6.367797


In [10]:
#create low spending flag with loc()
df_ords_prods_merge.loc[df_ords_prods_merge['users_avg_product_price'] < 10, 'Spending_flag'] = 'Low spender'

In [11]:
#create high spending flag with loc()
df_ords_prods_merge.loc[df_ords_prods_merge['users_avg_product_price'] >= 10, 'Spending_flag'] = 'High spender'

In [12]:
#check frequencies of new flag
df_ords_prods_merge['Spending_flag'].value_counts()

Low spender     31770614
High spender      634245
Name: Spending_flag, dtype: int64

### high spender count 634,245
### low spender count 31,770,614

In [13]:
#check columns used to create flag
df_ords_prods_merge[['user_id', 'users_avg_product_price', 'Spending_flag']].head(100)

,user_id,users_avg_product_price,Spending_flag
0,1,6.367797,Low spender
1,1,6.367797,Low spender
2,1,6.367797,Low spender
3,1,6.367797,Low spender
4,1,6.367797,Low spender
...,...,...,...
95,360,10.006250,High spender
96,377,8.496552,Low spender
97,387,7.396610,Low spender
98,420,7.387805,Low spender


In [14]:
#show all rows to view output
pd.options.display.max_rows = None
df_ords_prods_merge[['user_id', 'users_avg_product_price', 'Spending_flag']].head(100)

,user_id,users_avg_product_price,Spending_flag
0,1,6.367797,Low spender
1,1,6.367797,Low spender
2,1,6.367797,Low spender
3,1,6.367797,Low spender
4,1,6.367797,Low spender
5,1,6.367797,Low spender
6,1,6.367797,Low spender
7,1,6.367797,Low spender
8,1,6.367797,Low spender
9,1,6.367797,Low spender


## Create a frequency flag by grouping by user_id and finding median of days_last_prior_order

In [15]:
#create new column for median 
df_ords_prods_merge['median_days_since_last_order'] = 
    df_ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [16]:
#check output
df_ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,_exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,users_avg_product_price,Spending_flag,median_days_since_last_order
0,2539329,1,1,2,8,11.0,196,1,0,both,...,both,Mid-range product,Regularly busy,Regularly busy,average orders,10,New customer,6.367797,Low spender,20.0
1,2398795,1,2,3,7,15.0,196,1,1,both,...,both,Mid-range product,Regularly busy,Least busy days,average orders,10,New customer,6.367797,Low spender,20.0
2,473747,1,3,3,12,21.0,196,1,1,both,...,both,Mid-range product,Regularly busy,Least busy days,most orders,10,New customer,6.367797,Low spender,20.0
3,2254736,1,4,4,7,29.0,196,1,1,both,...,both,Mid-range product,Least busy,Least busy days,average orders,10,New customer,6.367797,Low spender,20.0
4,431534,1,5,4,15,28.0,196,1,1,both,...,both,Mid-range product,Least busy,Least busy days,most orders,10,New customer,6.367797,Low spender,20.0


In [17]:
#create frequency flag
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_last_order'] >20, 'frequency_flag'] = 'Non-frequent customer'

In [18]:
#create frequency flag
df_ords_prods_merge.loc[(df_ords_prods_merge['median_days_since_last_order'] <=20) &
                        (df_ords_prods_merge['median_days_since_last_order'] >10), 'frequency_flag'] = 'Regular customer'

In [19]:
#create frequency flag
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_last_order'] <=10, 'frequency_flag'] = 'Frequent customer'

In [20]:
#check columns used to create flag
df_ords_prods_merge[['median_days_since_last_order', 'frequency_flag']].head(25)

,median_days_since_last_order,frequency_flag
0,20.0,Regular customer
1,20.0,Regular customer
2,20.0,Regular customer
3,20.0,Regular customer
4,20.0,Regular customer
5,20.0,Regular customer
6,20.0,Regular customer
7,20.0,Regular customer
8,20.0,Regular customer
9,20.0,Regular customer


In [21]:
#check frequencies of frequency flag
df_ords_prods_merge['frequency_flag'].value_counts()

Frequent customer        20550144
Regular customer          9154455
Non-frequent customer     2700260
Name: frequency_flag, dtype: int64

### frequent customers = 20,550,144
### regular customers = 9,154,455
### non-frequent customers = 2,700,260
### most customers are considered frequent customers, place orders 10 or fewer days apart

## Export file

In [27]:
#export file to pkl
df_ords_prods_merge.to_pickle(os.path.join(path, '02.Data', 'Prepared_Data', 'orders_products_merged_4.8_task.pkl'))